In [2]:
import asyncio
from agents import Agent, Runner, OpenAIChatCompletionsModel, AsyncOpenAI
from openai.types.responses import ResponseTextDeltaEvent
import nest_asyncio
nest_asyncio.apply()  # This allows asyncio.run() to work in Jupyter
import os

# Ensure the API key is set in the environment (if needed)
os.environ["OPENAI_API_KEY"] = "lm-studio"

local_model = OpenAIChatCompletionsModel(
    model="qwen2.5-7b-instruct-1m@q8_0",
    openai_client=AsyncOpenAI(base_url="http://localhost:1234/v1"
    )
)

agent = Agent(
    name="Assistant",
    instructions="You're a helpful assistant",
    model=local_model,
)
async def stream_response():
    result = Runner.run_streamed(agent, "What is higher between 9.19 and 9.9?")
    # Use stream_events() instead of stream
    async for event in result.stream_events():
        # Handle raw response events for text streaming
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            print(event.data.delta, end="", flush=True)

# Now you can use asyncio.run() as normal
asyncio.run(stream_response())

The number 9.9 is higher than 9.19.

In [3]:
# we do need to reinitialize our runner before re-executing
response = Runner.run_streamed(
    starting_agent=agent,
    input="tell me a short story"
)

async for event in response.stream_events():
    if event.type == "raw_response_event" and \
        isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Sure! Here's a short story for you:

---

In the quiet village of Elderglen, nestled between rolling hills and ancient woods, lived an old blacksmith named Eli. His forge was a warm, inviting place filled with the scent of burning iron and the rhythmic clinking of hammers on metal. For years, he had crafted tools and weapons for the villagers—axeheads, horseshoes, and anything else they needed.

One rainy evening, as thunder rolled in from the hills, Eli heard a soft knock at his door. He opened it to find a young girl standing there, her eyes wide with fear. She was small for her age, wearing tattered clothes that clung to her wet skin. In her hands was a tiny, rusted key.

"Are you lost?" Eli asked, bending down to meet her gaze.

The girl nodded, clutching the key tighter. "I... I'm looking for someone," she whispered.

Eli guided her inside and offered her some hot tea. As they sat together in front of the fire, he learned that her name was Elara, and she had been wandering aimless

In [4]:
from agents import function_tool

@function_tool
def multiply(x: float, y: float) -> float:
    """Multiplies `x` and `y` to provide a precise
    answer."""
    return x*y

In [5]:
math_agent = agent.clone(
    name="Math Agent",
        instructions=(
        "You're a helpful assistant, remember to always "
        "use the provided tools whenever possible. Do not "
        "rely on your own knowledge too much and instead "
        "use your tools to help you answer queries."
    ),
    tools=[multiply]  # note that we expect a list of tools
)

In [6]:
response = Runner.run_streamed(
    starting_agent=agent,
    input="what is 7.814 multiplied by 103.909?"
)

async for event in response.stream_events():
    if event.type == "raw_response_event" and \
        isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

To find the product of 7.814 and 103.909, you simply multiply the two numbers together:

\[ 7.814 \times 103.909 = 812.699556 \]

So, the result is approximately 812.700 (rounded to three decimal places).

In [7]:
response = Runner.run_streamed(
    starting_agent=math_agent,
    input="what is 7.814 multiplied by 103.909?"
)

async for event in response.stream_events():
    if event.type == "raw_response_event" and \
        isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

7.814 multiplied by 103.909 equals 811.944926.

In [8]:
from openai.types.responses import (
    ResponseFunctionCallArgumentsDeltaEvent,  # tool call streaming
    ResponseCreatedEvent,  # start of new event like tool call or final answer
)

response = Runner.run_streamed(
    starting_agent=math_agent,
    input="what is 7.814 multiplied by 103.892?"
)

async for event in response.stream_events():
    if event.type == "raw_response_event":
        if isinstance(event.data, ResponseFunctionCallArgumentsDeltaEvent):
            # this is streamed parameters for our tool call
            print(event.data.delta, end="", flush=True)
        elif isinstance(event.data, ResponseTextDeltaEvent):
            # this is streamed final answer tokens
            print(event.data.delta, end="", flush=True)
    elif event.type == "agent_updated_stream_event":
        # this tells us which agent is currently in use
        print(f"> Current Agent: {event.new_agent.name}")
    elif event.type == "run_item_stream_event":
        # these are events containing info that we'd typically
        # stream out to a user or some downstream process
        if event.name == "tool_called":
            # this is the collection of our _full_ tool call after our tool
            # tokens have all been streamed
            print()
            print(f"> Tool Called, name: {event.item.raw_item.name}")
            print(f"> Tool Called, args: {event.item.raw_item.arguments}")
        elif event.name == "tool_output":
            # this is the response from our tool execution
            print(f"> Tool Output: {event.item.raw_item['output']}")

> Current Agent: Math Agent
{"x": 7.814, "y": 103.892}
> Tool Called, name: multiply
> Tool Called, args: {"x": 7.814, "y": 103.892}
> Tool Output: 811.812088
7.814 multiplied by 103.892 is approximately 811.8121 (rounded to four decimal places).